In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import emnist
from emnist import extract_training_samples, extract_test_samples

t_images, t_labels = extract_training_samples('digits')
v_images, v_labels = extract_test_samples('digits')

#print(t_images.shape)


transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,)),])
#transform = transforms.Compose([transforms.Normalize((0.1307,), (0.3081,)),])
#print(t_images.shape)
t_images = [t_images[i] for i in range(len(t_images))]
t_images = np.array(t_images)
#print(t_images.shape)
t_images = [np.array(transform(t_images[i])) for i in range(len(t_images))]
t_images = np.array(t_images)
print(t_images.shape)

(240000, 1, 28, 28)


In [2]:
#t_images = np.array(t_images)
#print(t_images[0])
#trainset = datasets.MNIST('PATH_TO_STORE_TRAINSET', download=True, train=True)#, transform=transform)
#valset = datasets.MNIST('PATH_TO_STORE_TESTSET', download=True, train=False)#, transform=transform)
#print(trainset[1])
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
#valloader = torch.utils.data.DataLoader(valset, batch_size=128, shuffle=True)

[[[-0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.42421296 -0.42421296]
  [-0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.42421296 -0.42421296 -0.3987565  -0.3987565
   -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.42421296 -0.42421296]
  [-0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.42421296 -0.42421296 -0.42421296 -0.42421296
   -0.42421296 -0.33511534  0.1612858   1.7268586   1.7395868
    0.55586106 -0.41148472 -0.42421296 -

   -0.42421296 -0.42421296 -0.42421296]]]


In [13]:
batch_size = 100
b_im = []
b_val = []
for i in range(len(t_images)//batch_size):
    b_im.append(t_images[i*batch_size:(i+1)*batch_size])
    b_val.append(t_labels[i*batch_size:(i+1)*batch_size])


b_im = np.array(b_im)
b_val = np.array(b_val)

print(b_im.shape)

#dataiter = iter(trainloader)
#images, labels = dataiter.next()

#print(images.shape)
#print(labels.shape)

(2400, 100, 1, 28, 28)


In [14]:
input_size = 784
hidden_sizes = [512, 256]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=10, bias=True)
  (5): LogSoftmax()
)


In [26]:
criterion = nn.NLLLoss()
#images, labels = next(iter(trainloader))
t_b_im = b_im[0]
print(t_b_im.shape)
t_b_im = torch.FloatTensor(t_b_im)
print(t_b_im.shape)
t_b_im = t_b_im.view(t_b_im.shape[0], -1)

print(t_b_im.shape)

logps = model(t_b_im) #log probabilities
print(logps.shape)
t_b_val = torch.LongTensor(b_val)
print(t_b_val[0].shape)
loss = criterion(logps, t_b_val[0]) #calculate the NLL loss

(100, 1, 28, 28)
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 10])
torch.Size([100])


IndexError: invalid index of a 0-dim tensor. Use tensor.item() to convert a 0-dim tensor to a Python number

In [29]:
optimizer = optim.SGD(model.parameters(), lr=0.004, momentum=0.9)
time0 = time()
epochs = 25
for e in range(epochs):
    running_loss = 0
    for images, labels in zip(b_im, b_val):
        # Flatten MNIST images into a 784 long vector
#         f_im = []
#         for i in range(batch_size):
#             k = []
#             for j in range(28):
#                 k += list(images[i][j])
#             f_im.append(k)
#         images = f_im
#         images = np.array(images)
#         trans = transforms.Compose([transforms.ToTensor()])
#         images = trans(images)
#         labels = torch.from_numpy(labels)
        images = torch.FloatTensor(images)
        #print(t_b_im.shape)
        images = images.view(images.shape[0], -1)
        #print(images.shape)
        # Training pass
        optimizer.zero_grad()
        output = model(images)
        
        labels = torch.LongTensor(labels)
        loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(t_images)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 0.0007653717748976002
Epoch 1 - Training loss: 0.0005163749136050076
Epoch 2 - Training loss: 0.000398329069869942
Epoch 3 - Training loss: 0.0003253897322584332
Epoch 4 - Training loss: 0.00027344296434651675
Epoch 5 - Training loss: 0.0002330566487435135
Epoch 6 - Training loss: 0.00020036222910857759
Epoch 7 - Training loss: 0.0001724865258295419
Epoch 8 - Training loss: 0.00014865588336566968
Epoch 9 - Training loss: 0.00012831365463074083
Epoch 10 - Training loss: 0.00011101553170907815
Epoch 11 - Training loss: 9.592829292099244e-05
Epoch 12 - Training loss: 8.305566710320515e-05
Epoch 13 - Training loss: 7.207946481254719e-05
Epoch 14 - Training loss: 6.234267856340618e-05
Epoch 15 - Training loss: 5.426554122010809e-05
Epoch 16 - Training loss: 4.7221810780380714e-05
Epoch 17 - Training loss: 4.125750026766279e-05
Epoch 18 - Training loss: 3.5954337038159183e-05
Epoch 19 - Training loss: 3.1504822114053844e-05
Epoch 20 - Training loss: 2.757352216587302

In [30]:
# def view_classify(img, ps):
#     ''' Function for viewing an image and it's predicted classes.
#     '''
#     ps = ps.data.numpy().squeeze()

#     fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
#     ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
#     ax1.axis('off')
#     ax2.barh(np.arange(10), ps)
#     ax2.set_aspect(0.1)
#     ax2.set_yticks(np.arange(10))
#     ax2.set_yticklabels(np.arange(10))
#     ax2.set_title('Class Probability')
#     ax2.set_xlim(0, 1.1)
#     plt.tight_layout()
    
# images, labels = next(iter(valloader))

# img = images[0].view(1, 784)
# with torch.no_grad():
#     logps = model(img)

# ps = torch.exp(logps)
# probab = list(ps.numpy()[0])
# print("Predicted Digit =", probab.index(max(probab)))
# view_classify(img.view(1, 28, 28), ps)

#transform = transforms.Compose([transforms.Normalize((0.1307,), (0.3081,)),])
#print(t_images.shape)
v_images = [v_images[i] for i in range(len(v_images))]
v_images = np.array(v_images)
#print(t_images.shape)
v_images = [np.array(transform(v_images[i])) for i in range(len(v_images))]
v_images = np.array(v_images)

In [37]:
batch_size = 100
v_im = []
v_val = []
for i in range(len(v_images)//batch_size):
    v_im.append(v_images[i*batch_size:(i+1)*batch_size])
    v_val.append(v_labels[i*batch_size:(i+1)*batch_size])


v_im = np.array(v_im)
v_val = np.array(v_val)

print(v_im.shape)
print(v_val.shape)

(400, 100, 1, 28, 28)
(400, 100)


In [49]:
correct_count, all_count = 0, 0
for images, labels in zip(v_im, v_val):
        # Flatten MNIST images into a 784 long vector
#         f_im = []
#         for i in range(batch_size):
#             k = []
#             for j in range(28):
#                 k += list(images[i][j])
#             f_im.append(k)
#         images = f_im
#         images = np.array(images)
#         trans = transforms.Compose([transforms.ToTensor()])
#         images = trans(images)
#         labels = torch.from_numpy(labels)
        images = torch.FloatTensor(images)
        #print(t_b_im.shape)
        images = images.view(images.shape[0], -1)
        #print(images.shape)
        # Training pass
        with torch.no_grad():
            logps = model(images)
        #print(logps.shape, logps[0])
        ps = torch.exp(logps)
        #print(ps.shape)
        for i in range(batch_size):
            prob_ans = ps[i]
            prob_ans = list(prob_ans)
            pred_label = prob_ans.index(max(prob_ans))
            #pred_label = ps.index(max(ps))
            true_label = labels[i]
            if(true_label == pred_label):
              correct_count += 1
            all_count += 1

In [51]:
print(all_count)
print(correct_count/all_count)

40000
0.99095


In [52]:
torch.save(model, './99percent_emnist_model_06102019.pt')